### Demo key shift with chopin prelude

Thoughts:
- turn original midi into running-status-free midi (avoiding confusion when recovering midi from smart contract call)
- turn midi into smart contracts
    - for note manipulation: `new note hex = original note hex + note_shift_amount * 2^8`; `length` unchanged

In [9]:
import mido
import sys
import binascii
import pprint
from collections import OrderedDict
from mido import Message, MidiFile, MidiTrack
from enum import Enum
from copy import copy, deepcopy

In [2]:
### Need to deal with variable length -- for both "len" of meta-event, and "delta-time"
### Need to deal with running status -- http://midi.teragonaudio.com/tech/midispec/run.htm

MSB_1 = ['8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

## TODO: ff00
## TODO: ff20, 21
## TODO: ff7f
## TODO: running status for all events

END_OF_TRACK = 'ff2f00'
TEXT_META_EVENTS = [f'ff0{i+1}' for i in range(9)]
TEMPO = 'ff5103'
SMPTE = 'ff5405'
TIME_SIG = 'ff5804'
KEY_SIG = 'ff5902'

class Track:
    def __init__(self, track):
        self.track = track
        self._parse()
    
    def _get_varlen_value (self, string):
        value = ''
        j = 0
        while True:
            if string[j] not in MSB_1: # reached last byte of variable-length field
                value = string[0:j+2]
                break
            else:
                j += 2
        return value
        
    def _varFieldToInt(self, hexstring):
        ret = 0
        current_byte = ''

        i=0
        while len(hexstring):
            current_byte = "0x" + hexstring[0:2]
            current_integer = int(current_byte, 16)
            ret = (ret << 7) | (current_integer & 0x7f)
            if not current_integer & 0x80:
                return ret
            hexstring = hexstring[2:]

            i += 1
            if i>1000:
                print('error')
                break
    
    def _parse(self):
        D = []
        
        assert self.track[0:8] == '4d54726b'
        self.chunklen = self.track[8:8+8]
        D.append({'type':'header', 'value':'4d54726b'})
        D.append({'type':'chunklen', 'value':self.track[8:8+8]})
        
        track = self.track[16:]
        i = 0
        state = 'delta-time' # state in ['delta-time', 'event']
        while True:
            if state == 'delta-time':
                value = self._get_varlen_value(track[i:])
                D.append( {'type':'delta time', 'value':value} )
                i += len(value)
                state = 'event'
                continue
                
            else: # state == 'event'
                running = False
                ## first decode event
                if track[i] not in MSB_1: # running status in use
                    event = last_event # for note on / note off / control change / program change - include channel n
                    running = True
                else:
                    track_first6 = track[i:i+6]
                    track_first4 = track[i:i+4]
                    track_first = track[i]
                    if track_first6 == END_OF_TRACK: # end of track; ff 2f 00
                        event = END_OF_TRACK
                    elif track_first6 == TIME_SIG: # ff 58 04 nn dd cc bb
                        event = TIME_SIG
                    elif track_first6 == KEY_SIG: # ff 59 02 sf mi
                        event = KEY_SIG
                    elif track_first6 == TEMPO: # ff 51 03 tt tt tt
                        event = TEMPO
                    elif track_first6 == SMPTE: # ff 54 05 hr mn se fr ff
                        event = SMPTE
                    elif track_first4 in TEXT_META_EVENTS:
                        event = track_first4
                    elif track_first in ['b','c','8','9']:
                        event = track[i:i+2]
                    
                ## then parse event and append to event list
                if event == END_OF_TRACK: # end of track; ff 2f 00
                    D.append( {'type':'end of track', 'type hex':END_OF_TRACK, 'value':''} )
                    break
                
                elif event == TIME_SIG: # ff 58 04 nn dd cc bb
                    D.append( {'type':'time signature', 'type hex':track_first6, 'value':track[i+6:i+6+8]} )
                    i += 14
                    
                elif event == KEY_SIG: # ff 59 02 sf mi
                    D.append( {'type':'key signature', 'type hex':track_first6, 'value':track[i+6:i+6+4]} )
                    i += 10
                
                elif event == TEMPO: # ff 51 03 tt tt tt
                    D.append( {'type':'tempo', 'type hex':track_first6, 'value':track[i+6:i+6+6]} )
                    i += 12
                
                elif event == SMPTE: # ff 54 05 hr mn se fr ff
                    D.append( {'type':'smpte', 'type hex':track_first6, 'value':track[i+6:i+6+10]} )
                    i += 16
                    
                # sequence/track name (ff 03 length text)
                # or marker (ff 06 length text)
                # or cue (ff 07 length text)
                elif event in TEXT_META_EVENTS:
                    if running:
                        name_length_varlen = self._get_varlen_value(track)
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + len(name_length_varlen) : i + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':event, 'omitted':2, ## omitted 2 bytes
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                    else:
                        name_length_varlen = self._get_varlen_value(track[i+4:])
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + 4 + len(name_length_varlen) : i + 4 + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':track_first4, 'omitted':0,
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += 4 + len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                
                # Assuming only bn/cn/8n/9n can use running-status
                elif event[0] == 'b': # control change; bn controller value
                    if running:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':1,
                                   'channel':event[1], 'controller':track[i:i+2], 'value':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':0,
                                   'channel':track[i+1], 'controller':track[i+2:i+4], 'value':track[i+4:i+6]} )
                        i += 6
                
                elif event[0] == 'c': # program change; cn program
                    if running:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':1,
                                   'channel':event[1], 'program':track[i:i+2]} )
                        i += 2
                    else:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':0,
                                   'channel':track[i+1], 'program':track[i+2:i+4]} )
                        i += 4
                
                elif event[0] == '9': # note on; 9n note velocity
                    if running:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                    
                elif track_first == '8': # note off; 8n note velocity
                    if running:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                
                last_event = event
                state = 'delta-time'
                
        self.segs = D    
    
    def segs_to_hexstring(self): # segs is list of events, each event is a dictionary
        hexstring = ''

        for event in self.segs:
            hexstring += Track.event_to_hexstring(event)
        return hexstring

    @staticmethod
    def event_to_hexstring(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex']*(not event['omitted']) + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['note'] + event['velocity']

        return hexstring
    
    @staticmethod
    def event_to_hexstring_discard_runningstatus(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex'] + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = event['type hex'] + event['channel'] + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = event['type hex'] + event['channel'] + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = event['type hex'] + event['channel'] + event['note'] + event['velocity']

        return hexstring
    
class Music:
    def __init__(self, name):
        self._parse_midi(name)
        self.name = name
    
    def _parse_midi(self, name):
        midi_file = f'./{name}.mid'

        with open(midi_file, 'rb') as f:
            hexdata = binascii.hexlify(f.read())
        hexdata_str = hexdata.decode("utf-8")

        MTrk = '4d54726b'
        tracks_str = []
        for i,chunk in enumerate(hexdata_str.split(MTrk)):
            if i == 0:
                header_str = chunk
            else:
                tracks_str.append (MTrk + chunk)

        track_objs = [Track(t) for t in tracks_str]
        
        self.header_str = header_str
        self.track_objs = track_objs

    def keyshift(self, shift_amount):
        track_objs = self.track_objs.copy()
        for obj in track_objs:
            for event in obj.segs:
                if event['type'] == 'note on' or event['type'] == 'note off':
                    note_hex = '0x'+event['note']
                    note_integer = int(note_hex, 16)
                    note_integer_shifted = note_integer + shift_amount
                    note_hex_shifted = hex(note_integer_shifted)[2:]
                    note_hex_shifted = note_hex_shifted.rjust(2, '0')
                    event['note'] = note_hex_shifted
        self.track_objs = track_objs
    
    def export_midi(self, name):
        recovered_tracks = [T.segs_to_hexstring() for T in self.track_objs]
        with open(f'{name}.mid', 'wb') as fout:
            fout.write( binascii.unhexlify(self.header_str + ''.join(recovered_tracks)) )

In [18]:
chpn_og = Music('chpn-p1')
print(f'number of music tracks: {len(chpn_og.track_objs)-1}')

number of music tracks: 6


In [25]:
for i,track_obj in enumerate(chpn_og.track_objs):
    omit = 0
    for event in track_obj.segs:
        if 'omitted' in event:
            omit += event['omitted']
    print(f'track {i} omitted {omit} bytes')

track 0 omitted 0 bytes
track 1 omitted 409 bytes
track 2 omitted 247 bytes
track 3 omitted 0 bytes
track 4 omitted 0 bytes
track 5 omitted 0 bytes
track 6 omitted 0 bytes


In [26]:
chpn = deepcopy(chpn_og)

In [27]:
print(chpn_og.track_objs[1].segs[1])
print(chpn_og.track_objs[2].segs[1])

{'type': 'chunklen', 'value': '00000519'}
{'type': 'chunklen', 'value': '00000430'}


In [28]:
chunklen1 = int('0x' + chpn.track_objs[1].segs[1]['value'], 16)
chunklen1_new = chunklen1 - 409
chunklen1_new_hex = hex(chunklen1_new)[2:]
chunklen1_new_hex = chunklen1_new_hex.rjust(8, '0')
chpn.track_objs[1].segs[1]['value'] = chunklen1_new_hex

In [29]:
chunklen2 = int('0x' + chpn.track_objs[2].segs[1]['value'], 16)
chunklen2_new = chunklen2 - 247
chunklen2_new_hex = hex(chunklen2_new)[2:]
chunklen2_new_hex = chunklen2_new_hex.rjust(8, '0')
chpn.track_objs[2].segs[1]['value'] = chunklen2_new_hex

In [30]:
print(chpn.track_objs[1].segs[1])
print(chpn.track_objs[2].segs[1])

{'type': 'chunklen', 'value': '00000380'}
{'type': 'chunklen', 'value': '00000339'}


In [47]:
def convert_hex_to_arr(hexstring):
        '''
        last element of the felt array is an integer indicating the hex-length of the last felt value
        '''
        arr = []
        s = hexstring
        while( len(s)>62 ):
            felt_hex = s[0:62]
            arr.append( int(felt_hex, 16) )
            s = s[62:]
        felt_hex = s
        last_length = len(felt_hex)
        arr.append( int(felt_hex, 16) )
        arr.append(last_length)
        
        return arr
    
def generate_single(hexstring, name):
        ret = []
        
        ret.append( '@view')
        ret.append(f'func {name} {{')
        ret.append( '        range_check_ptr')
        ret.append( '    } () -> (')
        ret.append( '        z_len : felt,')
        ret.append( '        z : felt*')
        ret.append( '    ):')
        ret.append( '    alloc_locals\n')
        ret.append( '    let (local z) = alloc()\n')
      
        arr = convert_hex_to_arr(hexstring)
        for i,num in enumerate(arr):
            ret.append(f'    assert [z+{i}] = {num}')
        ret.append(f'    let z_len = {i+1}')
        ret.append('\n    return (z_len, z)')
        ret.append('end')
        
        for line in ret:
            print(line)

def event_to_hexstring_no_omittance(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex'] + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = event['type hex'] + event['channel'] + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = event['type hex'] + event['channel'] + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = event['type hex'] + event['channel'] + event['note'] + event['velocity']

        return hexstring

### header

In [40]:
header_str = chpn.header_str
print(header_str)
print()
generate_single(header_str, 'header')

4d546864000000060001000701e0

@view
func header {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 1568433012465980210040715044389344
    assert [z+1] = 28
    let z_len = 2

    return (z_len, z)
end


In [98]:
'4d546864000000060001000701e0' == header_str

True

### tempo

In [41]:
tempo_track_str = chpn.track_objs[0].segs_to_hexstring()
generate_single(tempo_track_str, 'tempo')

@view
func tempo {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 136630055383401356149242572565537006397567730891369451453366224442570469166
    assert [z+1] = 56878481284227153506060204573840070485491661497866299408952274119203762226
    assert [z+2] = 57218742765747676653729478877327742453910538518085541581276926540566324000
    assert [z+3] = 119099539352645953382626715204872617300325554821710503471433969358017491828
    assert [z+4] = 179199866051856678441086820727855504111432963000120495484779184055806752110
    assert [z+5] = 182386416965249781959371082215309469022151687836411848675778097417160897582
    assert [z+6] = 86921838934465134510603573218541076527360967275041835275082931199162789898
    assert [z+7] = 1759974419771673291367073350716576123904882470560914998945739957575377269
    assert [z+8] = 179239919692890548195358917716852909526288841824410734111800313607068582914

In [99]:
'4d54726b00000e9c00ff031e43686f70696e205072656c75646520204e6f2e20312c204f70757320323800ff0222436f7079726967687420a92032303032206279204265726e64204b7275656765722000ff010f46726564657269632043686f70696e00ff01074167697461746f00ff011c46657274696767657374656c6c7420616d2032372e382e323030320a00ff01174e6f726d696572756e673a2032332e31322e323030320a00ff011555706461746520616d2032342e31322e323030320a00ff011355706461746520616d20322e362e323031300a00ff011455706461746520616d2032302e322e323031340a00ff011444617565723a20303a3334204d696e7574656e0a00ff5405600003000000ff580402030c0800ff5902000000ff51030d44bd28ff51030d2c4d18ff51030d14371cff51030cfc781cff51030ce50e18ff51030ccdf91cff51030cb7351cff51030ca0c218ff51030c8a9e10ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030c49f978ff51030c8a9e34ff51030c74c734ff51030c5f3b38ff51030c49f928ff51030c350008ff51030c240214ff51030c13340cff51030c586514ff51030c69f708ff51030c7c2b14ff51030c8e240cff51030ca05014ff51030cb32508ff51030cc5be14ff51030cd9030cff51030bd03264ff51030be3d514ff51030c350028ff51030c204e18ff51030c0be21cff51030bf7ba1cff51030be3d518ff51030bd0321cff51030bbccf1cff51030ba9ab14ff51030ba9ab14ff51030bbccf14ff51030bd03214ff51030be3d514ff51030bf7ba14ff51030c0be228ff51030b4d8564ff51030c0be214ff51030bbccf28ff51030ba9ab28ff51030b96c614ff51030b841d28ff51030b71b028ff51030b5f7e14ff51030b4d8514ff51030b5f7e14ff51030b71b014ff51030b841d14ff51030ba9ab14ff51030bbccf14ff51030bd03214ff51030bf7ba08ff51030ca0500cff51030af6e364ff51030b841d28ff51030b71b028ff51030b5f7e28ff51030b4d8528ff51030b3bc428ff51030b2a3b28ff51030b18e914ff51030b07cc14ff51030b18e914ff51030b3bc414ff51030b5f7e14ff51030b841d08ff51030bd0f90cff51030be17614ff51030b5f7e64ff51030bbccf3cff51030ba9ab28ff51030b96c63cff51030b841d28ff51030b71b028ff51030b5f7e08ff51030bb0ea0cff51030be17614ff51030bf1bb14ff51030c029414ff51030c133414ff51030c240214ff51030c356b14ff51030c469914ff51030c586514ff51030d541250ff51030cc5be30ff51030cb32528ff51030ca05034ff51030c8e2428ff51030c7c2b28ff51030c69f714ff51030c7c2b14ff51030c8e2408ff51030ca0500cff51030cb32508ff51030cc5be14ff51030cd9030cff51030cec0c08ff51030cffc620ff51030d90396cff51030d44bd0cff51030ddf2318ff51030dc47010ff51030daa220cff51030d90390cff51030d76b110ff51030d5d880cff51030d44bd0cff51030d2c4d1cff51030d14370cff51030cfc7810ff51030ce50e0cff51030ccdf90cff51030cb73510ff51030ca0c20cff51030c8a9e08ff51030c8a9e08ff51030ccdf90cff51030cfc7814ff51030d143708ff51030d2c4d14ff51030d44bd0cff51030d5d8808ff51030d76b114ff51030d90390cff51030daa22810cff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030bd03278ff51030c49f964ff51030c350050ff51030c204e1cff51030c240214ff51030c13340cff51030c586514ff51030c69f708ff51030c7c2b14ff51030c8e240cff51030ca05014ff51030cb32508ff51030cc5be14ff51030cd9030cff51030b841d78ff51030be3d5815cff51030be17614ff51030bf1bb08ff51030c029414ff51030c13340cff51030c240208ff51030c356b48ff51030b71b06cff51030c24020cff51030bbccf810cff51030ba9ab14ff51030bbccf28ff51030ba9ab08ff51030bf1bb20ff51030be17608ff51030bf1bb0cff51030c029408ff51030c13340cff51030c240208ff51030c356b0cff51030c469908ff51030c58650cff51030c69f708ff51030c7c2b0cff51030c8e2408ff51030ca0500cff51030b18e978ff51030bbccf64ff51030ba9ab64ff51030b96c614ff51030b96c614ff51030ba9ab14ff51030bbccf14ff51030bd03214ff51030be3d514ff51030bf7ba14ff51030c204e08ff51030ca0500cff51030b96c664ff51030c0be214ff51030ba9ab64ff51030b96c664ff51030b841d28ff51030ba9ab14ff51030bbccf28ff51030bd03214ff51030be3d514ff51030bf7ba08ff51030ca0500cff51030b2a3b56ff51030bd03250ff51030bbccf3cff51030ba9ab3cff51030b96c63cff51030b841d28ff51030b96c628ff51030ba9ab36ff51030c204e64ff51030c0be206ff51030b96c678ff51030b841d64ff51030b71b028ff51030b841d28ff51030b96c628ff51030ba9ab0eff51030bf7ba08ff51030ca0500cff51030b2a3b78ff51030ba9ab28ff51030b96c628ff51030b841d14ff51030b71b028ff51030b5f7e28ff51030b4d8514ff51030b3bc414ff51030b841d14ff51030b96c614ff51030ba9ab14ff51030bbccf14ff51030bd03214ff51030be3d514ff51030bf7ba08ff51030ca0500cff51030c0be278ff51030b71b028ff51030b5f7e28ff51030b4d8528ff51030b3bc428ff51030b2a3b28ff51030b18e914ff51030b07cc14ff51030b18e914ff51030b2a3b14ff51030b3bc414ff51030b4d8514ff51030b5f7e28ff51030b96c664ff51030b841d14ff51030ba9ab14ff51030b3bc428ff51030b2a3b28ff51030b18e928ff51030b07cc28ff51030af6e328ff51030ae62e14ff51030b07cc28ff51030b18e914ff51030b2a3b14ff51030b3bc428ff51030b96c678ff51030b2a3b3cff51030b18e928ff51030b07cc3cff51030af6e328ff51030ae62e28ff51030ad5ab14ff51030af6e314ff51030b07cc14ff51030b18e914ff51030b2a3b14ff51030b4d8514ff51030ca0c264ff51030dc47002ff51030e883d0cff51030e6af210ff51030e4e1c0cff51030e31b90cff51030dfa3f10ff51030ddf230cff51030dc4700cff51030daa2210ff51030d76b102ff51030d2c4d1cff51030d44bd0cff51030d5d881cff51030d76b10cff51030d90391cff51030daa220cff51030dc4700cff51030ddf2310ff51030d44bd34ff51030d5d8828ff51030d76b108ff51030d44bd14ff51030c8a9e78ff51030d5d8814ff51030d76b128ff51030d5d8814ff51030d44bd28ff51030d2c4d14ff51030d143728ff51030cfc7814ff51030ce50e28ff51030cfc7828ff51030d143750ff51030c49f964ff51030cb73514ff51030d76b128ff51030d5d8814ff51030d44bd28ff51030d2c4d14ff51030d143728ff51030cfc7814ff51030ce50e28ff51030ccdf914ff51030cb73514ff51030ce50e14ff51030d143714ff51030d44bd14ff51030d76b114ff51030daa2214ff51030dfa3f70ff51030dfa3f0cff51030e31b91cff51030e15c51cff51030dfa3f18ff51030ddf231cff51030dc4701cff51030daa2218ff51030d90391cff51030d76b10cff51030d903910ff51030daa2214ff51030dc47014ff51030ddf2314ff51030dfa3f14ff51030e15c514ff51030e31b928ff51030ee2fa6cff51030d44bd0cff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d13428118ff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030d44bd78ff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030ca0c278ff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030d44bd78ff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030d143778ff51030c8e2430ff51030c7c2b28ff51030c69f728ff51030c586528ff51030c469928ff51030c356b20ff51030c8e2414ff51030ca05008ff51030cb3250cff51030cc5be08ff51030cd9030cff51030cec0c08ff51030cffc614ff51030d134220ff51030d76b178ff51030e15c528ff51030dfa3f28ff51030ddf2328ff51030dc47028ff51030daa2228ff51030d903928ff51030d76b114ff51030daa2214ff51030ddf2314ff51030e15c514ff51030e4e1c14ff51030e883d14ff51030ee2fa64ff51030f02373cff51030ee2fa28ff51030ec43e28ff51030ea60028ff51030e883d28ff51030e6af228ff51030e4e1c14ff51030ea60014ff51030f023714ff51030f631314ff51030fc8f314ff510310343b14ff510310a55d78ff51031312d028ff510313463614ff5103137ab414ff510313b05328ff510313e71c14ff5103141f1914ff510314585528ff51031492db14ff510314ceb414ff5103150bee28ff5103154a9514ff5103158ab514ff510315cc5b28ff5103160f9714ff510316547714ff5103169b0928ff510316e36014ff5103172d8b14ff51031e84808740ff51031e848000ff2f00' == tempo_track_str

True

### music track one

In [132]:
def generate_single_note_manipulable(segs, name):
    i = 0
    ret = []
    ret.append('    local shift_amount_m256 = shift_amount * 256')
    for event in segs:
        if 'note' in event['type']:
            hexstr = Track.event_to_hexstring(event)
            hexstr_len = len(hexstr)
            ret.append( f'    assert [z+{i}] = {int(hexstr, 16)} + shift_amount_m256' )
            ret.append( f'    assert [z+{i+1}] = {hexstr_len}' )
            i += 2
        else:
            hexstr = Track.event_to_hexstring(event)
            s = hexstr
            while( len(s)>62 ):
                felt_hex = s[0:62]
                ret.append( f'    assert [z+{i}] = {int(felt_hex, 16)}' )
                ret.append( f'    assert [z+{i+1}] = 62' )
                s = s[62:]
                i += 2
            felt_hex = s
            last_length = len(felt_hex)
            ret.append( f'    assert [z+{i}] = {int(felt_hex, 16)}' )
            ret.append( f'    assert [z+{i+1}] = {last_length}' )
            i += 2
    for line in ret:
        print(line)

In [133]:
len(chpn_og.track_objs[1].segs)

832

In [134]:
generate_single_note_manipulable(chpn_og.track_objs[1].segs[0:416], '')

    local shift_amount_m256 = shift_amount * 256
    assert [z+0] = 1297379947
    assert [z+1] = 8
    assert [z+2] = 1305
    assert [z+3] = 8
    assert [z+4] = 0
    assert [z+5] = 2
    assert [z+6] = 5172255634965538127576624586516596
    assert [z+7] = 28
    assert [z+8] = 0
    assert [z+9] = 2
    assert [z+10] = 49152
    assert [z+11] = 4
    assert [z+12] = 0
    assert [z+13] = 2
    assert [z+14] = 11536228
    assert [z+15] = 6
    assert [z+16] = 0
    assert [z+17] = 2
    assert [z+18] = 2624
    assert [z+19] = 4
    assert [z+20] = 0
    assert [z+21] = 2
    assert [z+22] = 23423
    assert [z+23] = 4
    assert [z+24] = 0
    assert [z+25] = 2
    assert [z+26] = 450553776345030037904731835580182146853641395011731700227710265710693791032
    assert [z+27] = 62
    assert [z+28] = 926233139
    assert [z+29] = 8
    assert [z+30] = 33056
    assert [z+31] = 4
    assert [z+32] = 9452593 + shift_amount_m256
    assert [z+33] = 6
    assert [z+34] = 80
    assert [z

In [135]:
generate_single_note_manipulable(chpn_og.track_objs[1].segs[416:], '')

    local shift_amount_m256 = shift_amount * 256
    assert [z+0] = 0
    assert [z+1] = 2
    assert [z+2] = 16640 + shift_amount_m256
    assert [z+3] = 4
    assert [z+4] = 0
    assert [z+5] = 2
    assert [z+6] = 16993 + shift_amount_m256
    assert [z+7] = 4
    assert [z+8] = 120
    assert [z+9] = 2
    assert [z+10] = 18257 + shift_amount_m256
    assert [z+11] = 4
    assert [z+12] = 120
    assert [z+13] = 2
    assert [z+14] = 18176 + shift_amount_m256
    assert [z+15] = 4
    assert [z+16] = 0
    assert [z+17] = 2
    assert [z+18] = 20057 + shift_amount_m256
    assert [z+19] = 4
    assert [z+20] = 0
    assert [z+21] = 2
    assert [z+22] = 19014 + shift_amount_m256
    assert [z+23] = 4
    assert [z+24] = 80
    assert [z+25] = 2
    assert [z+26] = 18944 + shift_amount_m256
    assert [z+27] = 4
    assert [z+28] = 0
    assert [z+29] = 2
    assert [z+30] = 18260 + shift_amount_m256
    assert [z+31] = 4
    assert [z+32] = 80
    assert [z+33] = 2
    assert [z+3

### music track two

In [136]:
len(chpn_og.track_objs[2].segs)

640

In [137]:
generate_single_note_manipulable(chpn_og.track_objs[2].segs[0:320], '')

    local shift_amount_m256 = shift_amount * 256
    assert [z+0] = 1297379947
    assert [z+1] = 8
    assert [z+2] = 1072
    assert [z+3] = 8
    assert [z+4] = 0
    assert [z+5] = 2
    assert [z+6] = 20204122365158313696216914159220
    assert [z+7] = 26
    assert [z+8] = 0
    assert [z+9] = 2
    assert [z+10] = 49152
    assert [z+11] = 4
    assert [z+12] = 0
    assert [z+13] = 2
    assert [z+14] = 11536228
    assert [z+15] = 6
    assert [z+16] = 0
    assert [z+17] = 2
    assert [z+18] = 2624
    assert [z+19] = 4
    assert [z+20] = 0
    assert [z+21] = 2
    assert [z+22] = 23423
    assert [z+23] = 4
    assert [z+24] = 0
    assert [z+25] = 2
    assert [z+26] = 23423
    assert [z+27] = 4
    assert [z+28] = 0
    assert [z+29] = 2
    assert [z+30] = 16511
    assert [z+31] = 4
    assert [z+32] = 1
    assert [z+33] = 2
    assert [z+34] = 9446446 + shift_amount_m256
    assert [z+35] = 6
    assert [z+36] = 79
    assert [z+37] = 2
    assert [z+38] = 9216 + s

In [138]:
generate_single_note_manipulable(chpn_og.track_objs[2].segs[320:], '')

    local shift_amount_m256 = shift_amount * 256
    assert [z+0] = 0
    assert [z+1] = 2
    assert [z+2] = 15418 + shift_amount_m256
    assert [z+3] = 4
    assert [z+4] = 80
    assert [z+5] = 2
    assert [z+6] = 15360 + shift_amount_m256
    assert [z+7] = 4
    assert [z+8] = 33136
    assert [z+9] = 4
    assert [z+10] = 11099 + shift_amount_m256
    assert [z+11] = 4
    assert [z+12] = 10
    assert [z+13] = 2
    assert [z+14] = 11550720
    assert [z+15] = 6
    assert [z+16] = 36
    assert [z+17] = 2
    assert [z+18] = 16511
    assert [z+19] = 4
    assert [z+20] = 34
    assert [z+21] = 2
    assert [z+22] = 9448192 + shift_amount_m256
    assert [z+23] = 6
    assert [z+24] = 0
    assert [z+25] = 2
    assert [z+26] = 12366 + shift_amount_m256
    assert [z+27] = 4
    assert [z+28] = 80
    assert [z+29] = 2
    assert [z+30] = 12288 + shift_amount_m256
    assert [z+31] = 4
    assert [z+32] = 0
    assert [z+33] = 2
    assert [z+34] = 15421 + shift_amount_m256
 

### track three to end

In [139]:
hexstring = ''
#for track in chpn.track_objs[3:]:
for track in chpn.track_objs[3:]:
    for event in track.segs:
        hexstring += event_to_hexstring_no_omittance(event)

In [140]:
generate_single(hexstring, 'music_3to6')

@view
func music_3to6 {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 136630055383401265450026481975897785629628796645206062477878610220790214446
    assert [z+1] = 86799204449262242841506209475406639191495463763289587680747166378917830979
    assert [z+2] = 196896293983749844085133960720503419841174072837427151471105177045055137637
    assert [z+3] = 201427445120937909719305475177723715238454465423221075366242329356546369889
    assert [z+4] = 195120495690111691153414471557661700819694411966667018762683761288135930985
    assert [z+5] = 9706972061181884266817624211700865765945088
    assert [z+6] = 36
    let z_len = 7

    return (z_len, z)
end
